In [112]:
import pandas as pd
import pyarrow


In [122]:
data_0 = pd.read_parquet('/Users/elireadnour/Documents/Computer/Classes/machineLearning/MLFinal/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet')
columns_needed = ['date_id', 'time_id', 'symbol_id', 'weight', 'feature_00', 'feature_01',
       'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06',
       'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11',
       'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16',
       'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26',
       'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31',
       'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36',
       'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41',
       'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46',
       'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51',
       'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56',
       'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61',
       'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66',
       'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feature_71',
       'feature_72', 'feature_73', 'feature_74', 'feature_75', 'feature_76',
       'feature_77', 'feature_78','responder_6', 'responder_6_lag_1']
lags = pd.read_parquet('/Users/elireadnour/Documents/Computer/Classes/machineLearning/MLFinal/jane-street-real-time-market-data-forecasting/lags.parquet/date_id=0/part-0.parquet')


In [123]:
merged_df = pd.merge(data_0, lags, on='symbol_id', how='left')
merged_df.dropna(axis=1, inplace= True)


Split Data

In [124]:
X = merged_df.drop('responder_6', axis=1)
y = merged_df['responder_6']

In [126]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the XGBoost Regressor
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model using Mean Squared Error (or other relevant metrics)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (XGBoost): {mse}")


Mean Squared Error (XGBoost): 0.14514872431755066


In [127]:
from sklearn.model_selection import cross_val_score

# Using 5-fold cross-validation for model validation
kf_scores = cross_val_score(xgb_model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Average Cross-Validation MSE: {-kf_scores.mean()}")


Average Cross-Validation MSE: 0.16485415697097777


In [129]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5, 7],
}

random_search = RandomizedSearchCV(XGBRegressor(objective='reg:squarederror'), param_distributions=param_dist, n_iter=20, cv=2, scoring='neg_mean_squared_error', verbose=2, random_state=42)
random_search.fit(X_train, y_train)

print(f"Best Parameters: {random_search.best_params_}")


Fitting 2 folds for each of 20 candidates, totalling 40 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   5.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   5.1s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, min_child_weight=3, n_estimators=300, subsample=0.8; total time=   7.6s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, min_child_weight=3, n_estimators=300, subsample=0.8; total time=   7.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=5, min_child_weight=3, n_estimators=500, subsample=0.8; total time=  11.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=5, min_child_weight=3, n_estimators=500, subsample=0.8; total time=  12.1s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=300, subsample=0.8; total t

Tuned XGB Model using parameters from Randomized Search CV

In [130]:
from xgboost import XGBRegressor

# Train the final model with the best parameters
final_xgb_model = XGBRegressor(
    objective='reg:squarederror',
    subsample=1.0,
    n_estimators=500,
    min_child_weight=3,
    max_depth=10,
    learning_rate=0.1,
    colsample_bytree=0.6,
    random_state=42
)

# Fit on the training data
final_xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [131]:
from sklearn.metrics import mean_squared_error

# Make predictions on the test set
y_pred = final_xgb_model.predict(X_test)

# Calculate Mean Squared Error on the test set
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (Final Model): {mse}")


Mean Squared Error (Final Model): 0.10424154996871948


In [ ]:
from sklearn.model_selection import cross_val_score

# Using 5-fold cross-validation for model validation
kf_scores = cross_val_score(xgb, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Average Cross-Validation MSE: {-kf_scores.mean()}")


# Data Preprocessing

Data Preprocessing: Standard Scaller 

In [117]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

X = scale.fit_transform(X)



PCA

In [118]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)  # Automaticall collect enough features to caputre 95 % of variance
X_pca = pca.fit_transform(X)

cumulative_variance = pca.explained_variance_ratio_.cumsum()
print(cumulative_variance)


[0.10730664 0.20812827 0.27430323 0.3348844  0.38985676 0.44306043
 0.4930155  0.5316344  0.5646317  0.5944145  0.62315035 0.6516101
 0.67846054 0.70308244 0.7267795  0.7499029  0.7708257  0.7903909
 0.8078895  0.8237381  0.8393095  0.8532956  0.86696416 0.88004553
 0.8927529  0.9019354  0.9109082  0.9195607  0.92790073 0.9355657
 0.9425222  0.9482943  0.9533449 ]


XGB w/PCA

In [119]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Initialize the XGBoost Regressor
xgb_model1 = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

# Train the model
xgb_model1.fit(X_train1, y_train1)

# Predict on the test set
y_pred1 = xgb_model1.predict(X_test1)

# Evaluate the model using Mean Squared Error (or other relevant metrics)
mse1 = mean_squared_error(y_test1, y_pred1)
print(f"Mean Squared Error (XGBoost): {mse1}")


Mean Squared Error (XGBoost): 0.32325002551078796


In [ ]:
from sklearn.model_selection import cross_val_score

# Using 5-fold cross-validation for model validation
kf_scores = cross_val_score(xgb_model, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
print(f"Average Cross-Validation MSE: {-kf_scores.mean()}")


Data Preprocessing
- Heat maps Variance
- XGBoost --> Finds most highly correlated data

Random Forest

Data Preprocessing
- Heat maps Variance
- XGBoost --> Finds most highly correlated data

Random Forest